# Introduction

This case study is for the dataset from telecom compony that provide phone and internet service.
The goal of this case is to analyze customer data to predict their hebavior(leave or stay) and develop model to retain customers.

# 1. Ask

We need to define our main task before this case stady.
#### 1.1 Business Task
Firstly, we set down the questions that we expect to ge answer:
1. What product features will cause customers to leave?
2. How to use the findings from the dataset to help the compony retain customers?

#### 1.2 Main StickHolder
1. Manergenment
2. Marketing tenm
3. Operation team

#### 1.3 Deliverables
1. A clear summary of the business task.
2. Documentation of any cleaning or manipulation of dataset.
3. A summary of analysis.
4. Supporting visualizations and key findings.
5. Recommendations based on the analysis.

# 2. Prepare
#### 2.1 Information of Data Source
1. The data source is available on [Kaggle](https://www.kaggle.com/blastchar/telco-customer-churn)
2. Generated by IBM Samples Team.
3. Data content
**Each row represents a customer, each column contains customer’s attributes described on the column Metadata.**
> The data set includes information about:
*  Customers who left within the last month – the column is called Churn.
*  Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies.
*  Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges.
*  Demographic info about customers – gender, age range, and if they have partners and dependents.

# 3. Process
Using Pyhton3 to prepare and process dataset.

#### 3.1 Prepare Libraries and Import Dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# read data
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

#### 3.2 Data cleaning and manipulation

1. > Observe and familiarize with data
2. > Check for null or missing values
3. > Perform sanity check of data

 **3.2.1 Data Exploration**

***Which features are avalaible in this dataset?***

In [ ]:
df.columns.values

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Feature Types
def df_summary(df):
    
    '''
    input the dataframe, and it will return a summary table with columns data type 
    
    '''
    
    #create a dataframe call summary
    summary = pd.DataFrame(df.dtypes, columns=['dtype'])
    
    #Number of missing values(-1 count)
    summary['num_missing'] = df.isna().sum().values
    
    #Number of unique values by features
    summary['num_uniques'] = df.nunique().values
    
    return summary
summary = df_summary(df)
summary

In [ ]:
print("Ubique values of each columns: ")
for col in df.columns:
    print(f"{col}: \n{df[col].unique()}\n")

In [ ]:
df.describe()

***Which features are numerical?***

tenure, MonthlyCharges, TotalCharges

***Observation for data exploration***

1. TotalCharges is wrongly classified as object dtype and has to be converted to float64 dtype.
2. As SeniorCitizen is a True-False item, we would like to convert 1, 0 to Yes, No.
3. There is not N/A values in this dataset, but we still need to check if there is blank string in some columns.
4. The numeric type of tenure is by monthly, we can consider convert it to yearly for analysis.
5. The average tenure is 32.4 months, and the average monthly charges is 64.76 dollors, 25 percent of customers are paying more than $89.85 per month

***3.2.2 Data Converting***

Firstly, we need to convert the TotalCharges to float64 dtype. 
The replace the SenniorCitizen to 1 or 0. 
Finally check the blank string in dataset.

In [ ]:
# Copy raw data to keep raw data source
df_data = df.copy()

In [ ]:
# Convert TotalCharges
df_data['TotalCharges'] = pd.to_numeric(df_data['TotalCharges'], errors='coerce')
df_data["SeniorCitizen"].replace(to_replace = 1, value = "Yes", inplace =True)
df_data["SeniorCitizen"].replace(to_replace = 0, value = "No", inplace =True)


#another way to convert
# df_data["SeniorCitizen"] = np.where(df_data["SeniorCitizen"] == "Yes", 1,0)
df_data.isna().sum()

In [ ]:
# checking the N/A data
df_data[df_data['TotalCharges'].isna() == True]

In [ ]:
# No more information for N/A data, so we will drop it.
df_data.dropna(inplace = True)
# customerID no use, we will drop it.
df_data.drop(columns=["customerID"], inplace=True)
'''
# tenure transfer to Yearly
df_data['tenure'].max()
labels = [f"{i} - {i+11}" for i in range(1,72,12)]
df_data['tenure_group'] = pd.cut(df_data.tenure, range(1,80,12), right=False, labels=labels)
df_data['tenure_group'].value_counts()
'''

In [ ]:
df_data.head()

> Data exploration, cleaning and manipulation is completed, data is ready to be analysed

In [ ]:
'''
def uni_plot(categorical_list):
    df_categoric = df.loc[:, categorical_list]
    for i in categorical_list:
        plt.figure()
        plt.figure(figsize=(10,8))
        ax = sns.countplot(x=i, data = df_categoric)
        plt.title(f"Distribution of {i}")
        
        # add data text
        for p in ax.patches:
            #get hirht of each bar
            height = p.get_height()
            #add text to each bar
            ax.text(x=p.get_x() + (p.get_width()/2), # x-coordinate position of data label, padded to be in the middle of the bar
                    y=height+10, # y-coordinate position of data label, padded 100 above bar
                    s = "{:.0f}".format(height), # data label, formatted to ignore decimals
                    ha = 'center') # sets horizontal alignment (ha) to center
                    
plt.figure(figsize=(10,8))
sns.countplot(df_data['tenure_group'], hue=df_data['Churn'])
plt.title(f"Distribution of tenure_group")

def category_plot(categorical_list):
    df_categoric = df.loc[:, categorical_list]
    for i in categorical_list:
        plt.figure()
        plt.figure(figsize=(10,8))
        ax = sns.countplot(x=i, data = df_categoric, hue = "Churn")
        plt.title(f"{i} in term of Churn")
        
        # add data text
        for p in ax.patches:
            #get hirht of each bar
            height = p.get_height()
            #add text to each bar
            ax.text(x=p.get_x() + (p.get_width()/2), # x-coordinate position of data label, padded to be in the middle of the bar
                    y=height+10, # y-coordinate position of data label, padded 100 above bar
                    s = "{:.0f}".format(height), # data label, formatted to ignore decimals
                    ha = 'center') # sets horizontal alignment (ha) to center

'''

# 4. Analysis
#### 4.1 Explore the distribution of features

***Distribution of numerical feature values***

In [ ]:
df_data.describe()

In [ ]:
fig, axes = plt.subplots(1,3,figsize=(10,4))
plt.suptitle("Distribution of numerical feature values", fontsize = 20)
cols = ["tenure", "MonthlyCharges", "TotalCharges"]
ylabel = ["Months", "Dollars", "Dollars"]
for i in range(3):
    sns.violinplot(data=df_data[cols[i]], orient="v", ax= axes[i],  cut=0)
    axes[i].grid(linestyle="--", alpha=0.5)
    axes[i].set_xticks(np.arange(1))
    axes[i].set_ylabel(ylabel[i], fontsize=15)
label1 = ["tenure"]
label2 = ["MonthlyCharges"]
label3 = ["TotalCharges"]
axes[0].set_xticklabels(label1, fontsize=15)
axes[1].set_xticklabels(label2, fontsize=15)
axes[2].set_xticklabels(label3, fontsize=15)
fig.tight_layout()
plt.show()

***Observation***
* Totla sample are 7032.
* The distribution of tenure is relatively scattered. The mode is 1 month(8.7%).
* The TotalCharges range of top 25% is wide, with a maximum of 8684 dollars, but half of the users spend less than 1397 dollars.

***The Distribution of categorical features***

In [ ]:
df_data.describe(include=['O'])

**Demographic infomation**
* Customers are 50.5% male and 49.5% female.
* 70.2% customers don't have dependents.

**Service**
* There are 42% users using MultipleLines service.
* 78.4% customers signed up the iternet service, and the most popular internet service item is StreamingMovies with 38.8%.

**Customer account Info**
* More than half of users use Month-to_Month contract(55%).
* The Churn rate is 26.6%.



#### 4.2 Correlation features to Churn

**Each feature proportion for the Churn**

As we want to know each feature correlate with churn, we will divide customers into 2 groups

In [ ]:
df_data["Churn"].replace(to_replace = "Yes", value = 1, inplace = True)
df_data["Churn"].replace(to_replace = "No", value = 0, inplace = True)
df_data = df_data.iloc[:, 0:20]

#draw bar plot
cate_cols = ['SeniorCitizen', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
colors = ["Salmon", "#5F9EA0"]

fig, axes = plt.subplots(nrows = 8,ncols = 2,figsize = (15,40))

for i,feature in enumerate(cate_cols):
    if i < 8:
        ax = df_data.groupby([feature,'Churn']).size().groupby(level=0).apply( lambda x: 100 * x / x.sum()).unstack().plot(kind='bar', color=colors, ax=axes[i,0])
    elif i >=8 and i < 16:
        ax = df_data.groupby([feature,'Churn']).size().groupby(level=0).apply( lambda x: 100 * x / x.sum()).unstack().plot(kind='bar', color=colors, ax=axes[i-8,1])
    
    ax.set_title("Distribution of {} and % Churn".format(feature), fontsize = 20)
    ax.set_ylabel("Percentage", fontsize=15)
    ax.set_xlabel("{}".format(feature), fontsize=15)
    ax.legend(["No", "Yes"], fontsize=12)
    
    for patch in ax.patches:
        width, height = patch.get_width(), patch.get_height()
        ax.annotate('{:.0f}%'.format(height), (patch.get_x()+0.1*width, patch.get_y()+0.5*height),
                    color = 'Black',
                    weight = 'bold',
                    size = 15)

    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    plt.setp(ax.get_xticklabels(), rotation=10, fontsize=12)

plt.tight_layout()
plt.show()

df_data.head()

**Demographic info**
* Senior citizen have a much higher rate: 42%.
* Customer without partner or dependents are more likely to leave.

**Service**
* Customer with Fiber optic are very liekly to leave.
* Customer who does not sign up for OnlineSecurity/OnlineBackup/DeviceProtection/TechSupport are very likely to churn.

**Customer account info**
* Customer with paperless billing or use electronic payment is likely to leave.
* The shorter the contract, the higher the customer churn rate.

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (15,4))
ax[0].set_title("Distribution of tenure by Churn", fontsize = 20)
item = df_data['Churn'].unique()
sns.kdeplot(df_data.tenure[(df_data["Churn"] == item[0]) ], color = "Gold", shade = True, ax = ax[0])
sns.kdeplot(df_data.tenure[(df_data["Churn"] == item[1]) ], ax = ax[0], color = "#5F9EA0", shade = True)
ax[0].legend(["{}".format(item[0]), "{}".format(item[1])], loc='upper right')
ax[0].set_ylabel('Density')
ax[0].set_xlabel('tenure')

sns.lineplot(x = "tenure", y = "Churn", data = df_data, ax = ax[1])
ax[1].set_title("Churn rate of tenure", fontsize = 20)
plt.show()

df_data.head()

* There's a peak in distribution of customers who left, but the distribution curve of number of customers who stay is stable.
* There is a trend that the larger the tenure, the smaller the churn rate.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 4))
ax[0].set_title("Distribution of Monthly Charges by Churn", fontsize = 20)
item = df_data["Churn"].unique()
sns.kdeplot(df_data.MonthlyCharges[(df_data["Churn"] == item[0]) ], color="Gold", shade = True, ax = ax[0])
ax[0] = sns.kdeplot(df_data.MonthlyCharges[(df_data["Churn"] == item[1]) ], ax = ax[0], color="#5F9EA0", shade= True)
ax[0].legend(["{}".format(item[0]), "{}".format(item[1])], loc='upper right')
ax[0].set_ylabel('Density')
ax[0].set_xlabel('Total Charges')

df_data["MonthlyCharges_cut"] = pd.cut(df_data.MonthlyCharges, bins=15, labels=np.arange(15))
sns.lineplot(x = "MonthlyCharges_cut", y = "Churn", data = df_data, ax = ax[1])
ax[1].set_title("Churn rate of MonthlyCharges", fontsize = 20)
plt.show()

* Most customers don’t spend too little every month, but 2 groups have different peak values. Peak - Churn: around 80 dollars, Not Churn: around 20 dollars.
* The churn rate and monthly charges are not clearly related to each other.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 4))
ax[0].set_title("Distribution of Total Charges by Churn", fontsize = 20)
item = df_data["Churn"].unique()
sns.kdeplot(df_data.TotalCharges[(df_data["Churn"] == item[0]) ], color="Gold", shade = True, ax = ax[0])
ax[0] = sns.kdeplot(df_data.TotalCharges[(df_data["Churn"] == item[1]) ], ax = ax[0], color="#5F9EA0", shade= True)
ax[0].legend(["{}".format(item[0]), "{}".format(item[1])], loc='upper right')
ax[0].set_ylabel('Density')
ax[0].set_xlabel('Total Charges')

df_data["TotalCharges_cut"] = pd.cut(df_data.TotalCharges, bins=9, labels=np.arange(9))
sns.lineplot(x = "TotalCharges_cut", y = "Churn", data = df_data, ax = ax[1])
ax[1].set_title("Churn rate of TotalCharges", fontsize = 20)
plt.show()

* The total charges distribution of customers who left is tightly clustered around the peak, and the general distribution of customers who stay is similar as it.
* There is a trend that the larger the TotalCharges, the smaller the churn rate.

***The correlation between category features and churn***

In [ ]:
corlor = ["Salmon", "#5F9EA0"]
sns.catplot(x="Partner", y="Churn",
           hue="Dependents", col="SeniorCitizen",
           data=df_data, kind="point",
           dodge=True, palette=colors,
           height=3, aspect=1.5)

* Senior citizen who don't have partner and dependents are most likely to leave, the churn rate about 50%.

***Different categoris of people use service***

In [ ]:
cols = ["OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies"]
colors = ["Salmon", "#5F9EA0"]
fig, ax = plt.subplots(3, 2, figsize=(15,12))
df1 = pd.melt(df_data[(df_data["InternetService"] != "No") & (df_data["Partner"] =="Yes")][cols]).rename({'value': 'Has service'}, axis=1)
sns.countplot(data=df1, x = 'variable', hue='Has service', ax = ax[0, 0], palette = colors)

df2 = pd.melt(df_data[(df_data["InternetService"] != "No") & (df_data["Partner"] =="No")][cols]).rename({'value': 'Has service'}, axis=1)
sns.countplot(data=df2, x='variable', hue='Has service', ax = ax[0, 1], palette = colors)
df3 = pd.melt(df_data[(df_data["InternetService"] != "No") & (df_data["Dependents"] =="Yes")][cols]).rename({'value': 'Has service'}, axis=1)
sns.countplot(data=df3, x='variable', hue='Has service', ax = ax[1, 0], palette = colors)
df4 = pd.melt(df_data[(df_data["InternetService"] != "No") & (df_data["Dependents"] =="No")][cols]).rename({'value': 'Has service'}, axis=1)
sns.countplot(data=df4, x='variable', hue='Has service', ax = ax[1, 1], palette = colors)
df5 = pd.melt(df_data[(df_data["InternetService"] != "No") & (df_data["SeniorCitizen"] =="Yes")][cols]).rename({'value': 'Has service'}, axis=1)
sns.countplot(data=df5, x='variable', hue='Has service', ax = ax[2, 0], palette = colors)
df6 = pd.melt(df_data[(df_data["InternetService"] != "No") & (df_data["SeniorCitizen"] =="No")][cols]).rename({'value': 'Has service'}, axis=1)
sns.countplot(data=df6, x='variable', hue='Has service', ax = ax[2, 1], palette = colors)

for i in range(3):
    for j in range(2):
        ax[i,j].set_xlabel('Service Item', fontsize = 15)
        ax[i,j].set_ylabel('Num of customers',  fontsize = 15)
        ax[i,j].legend(bbox_to_anchor=(1.0, 1.0), loc='upper left', title="Has service")
        plt.setp(ax[i,j].get_xticklabels(), rotation=15, fontsize=12)

ax[0,0].set_title("Partner = Yes", fontsize = 20)
ax[0,1].set_title("Partner = No", fontsize = 20)
ax[1,0].set_title("Dependents = Yes", fontsize = 20)
ax[1,1].set_title("Dependents = No", fontsize = 20)
ax[2,0].set_title("SeniorCitizen = Yes", fontsize = 20)
ax[2,1].set_title("SeniorCitizen = No", fontsize = 20)
plt.subplots_adjust(wspace=0.4, hspace=0.7)
plt.show()

df_data.head()

* Most customers without a partner or dependents do not sign up for additional services.
* Most senior citizen customers do not sign up for additional services, too.

***Correlation between features and MonthlyCharges***

In [ ]:
col_name1 = ["SeniorCitizen", "Partner", "Dependents"]

fig, ax = plt.subplots(2, 2, figsize=(15,10))
print("Average:")
    
for i,j in enumerate(col_name1):
    item = df_data[j].unique()
    if i < 2:
        ax[0,i] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[0]) ], color="Gold", shade = True, ax = ax[0,i])
        ax[0,i] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[1]) ], ax =ax[0,i], color="#5F9EA0", shade= True)
        ax[0,i].set_title("Distribution of monthly charges by {}".format(j), fontsize = 20)
        ax[0,i].legend(["{}".format(item[0]), "{}".format(item[1])], loc='upper right')
        ax[0,i].set_ylabel('Density', fontsize = 15)
        ax[0,i].set_xlabel('Monthly Charges', fontsize = 15)
    elif i >= 2:
        ax[1,i-2] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[0]) ], color="Gold", shade = True, ax = ax[1,i-2])
        ax[1,i-2] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[1]) ], ax =ax[1,i-2], color="#5F9EA0", shade= True)
        ax[1,i-2].set_title("Distribution of monthly charges by {}".format(j), fontsize = 20)
        ax[1,i-2].legend(["{}".format(item[0]), "{}".format(item[1])], loc='upper right')
        ax[1,i-2].set_ylabel('Density', fontsize = 15)
        ax[1,i-2].set_xlabel('Monthly Charges', fontsize = 15)
    ave1 = df_data[df_data[j]==item[0]].MonthlyCharges.mean()
    ave2 = df_data[df_data[j]==item[1]].MonthlyCharges.mean()
    print(f"{j} ({item[0]}/{item[1]}) -->  {ave1} / {ave2}.")

fig.delaxes(ax[1,1])
plt.tight_layout()
plt.show()

In [ ]:
col_name3 = ["StreamingTV", "Contract", "StreamingMovies", "InternetService"]

fig, ax = plt.subplots(2, 2, figsize=(15,10))
print("Average:")
for i,j in enumerate(col_name3):
    if i < 2:
        ax[i,0].set_title("Distribution of monthly charges by {}".format(j), fontsize = 20)
        item = df_data[j].unique()
        ax[i,0] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[0]) ], color="Salmon", shade = True, ax = ax[i,0])
        ax[i,0] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[1]) ], ax =ax[i,0], color="Gold", shade= True)
        ax[i,0] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[2]) ], ax =ax[i,0], color="#5F9EA0", shade= True)
        ax[i,0].legend(["{}".format(item[0]), "{}".format(item[1]), "{}".format(item[2])], loc='upper right')
        ax[i,0].set_ylabel('Density')
        ax[i,0].set_xlabel('Monthly Charges')
        if j != "Contract":
            ax[i,0].set_ylim(0,0.12)
    elif i >=2:
        ax[i-2,1].set_title("Distribution of monthly charges by {}".format(j), fontsize = 20)
        item = df_data[j].unique()
        ax[i-2,1] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[0]) ], color="Salmon", shade = True, ax = ax[i-2,1])
        ax[i-2,1] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[1]) ], ax =ax[i-2,1], color="Gold", shade= True)
        ax[i-2,1] = sns.kdeplot(df_data.MonthlyCharges[(df_data[j] == item[2]) ], ax =ax[i-2,1], color="#5F9EA0", shade= True)
        ax[i-2,1].legend(["{}".format(item[0]), "{}".format(item[1]), "{}".format(item[2])], loc='upper right')
        ax[i-2,1].set_ylabel('Density')
        ax[i-2,1].set_xlabel('Monthly Charges')
        if j != "Contract":
            ax[i-2,1].set_ylim(0,0.12)
    ave1 = df_data[df_data[j]==item[0]].MonthlyCharges.mean()
    ave2 = df_data[df_data[j]==item[1]].MonthlyCharges.mean()
    ave3 = df_data[df_data[j]==item[2]].MonthlyCharges.mean()
    print(f"{j} ({item[0]}/ {item[1]}/ {item[2]}) -->  {ave1} / {ave2} / {ave3}.")

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,5))
print("Average:")

ax[0].set_title("Distribution of monthly charges by PaperlessBilling", fontsize = 20)
item = df_data["PaperlessBilling"].unique()
ax[0] = sns.kdeplot(df_data.MonthlyCharges[(df_data["PaperlessBilling"] == item[0]) ], color="Gold", shade = True, ax = ax[0])
ax[0] = sns.kdeplot(df_data.MonthlyCharges[(df_data["PaperlessBilling"] == item[1]) ], ax = ax[0], color="#5F9EA0", shade= True)
ax[0].legend(["{}".format(item[0]), "{}".format(item[1])], loc='upper right')
ax[0].set_ylabel('Density')
ax[0].set_xlabel('Monthly Charges')


ave1 = df_data[df_data["PaperlessBilling"]==item[0]].MonthlyCharges.mean()
ave2 = df_data[df_data["PaperlessBilling"]==item[1]].MonthlyCharges.mean()
print(f"PaperlessBilling ({item[0]}/{item[1]}) -->  {ave1} / {ave2}.")

ax[1].set_title("Distribution of monthly charges by PaymentMethod", fontsize = 20)
item = df_data["PaymentMethod"].unique()
ax[1] = sns.kdeplot(df_data.MonthlyCharges[(df_data["PaymentMethod"] == item[0]) ], color="Salmon", shade = True, ax = ax[1])
ax[1] = sns.kdeplot(df_data.MonthlyCharges[(df_data["PaymentMethod"] == item[1]) ], ax = ax[1], color="Gold", shade= True)
ax[1] = sns.kdeplot(df_data.MonthlyCharges[(df_data["PaymentMethod"] == item[2]) ], ax = ax[1], color="#5F9EA0", shade= True)
ax[1] = sns.kdeplot(df_data.MonthlyCharges[(df_data["PaymentMethod"] == item[3]) ], ax = ax[1], color="#D3D3D3", shade= True)
ax[1].legend(["{}".format(item[0]), "{}".format(item[1]), "{}".format(item[2]), "{}".format(item[3])], loc='upper right')
ax[1].set_ylabel('Density')
ax[1].set_xlabel('Monthly Charges')

ave1 = df_data[df_data["PaymentMethod"]==item[0]].MonthlyCharges.mean()
ave2 = df_data[df_data["PaymentMethod"]==item[1]].MonthlyCharges.mean()
ave3 = df_data[df_data["PaymentMethod"]==item[2]].MonthlyCharges.mean()
ave4 = df_data[df_data["PaymentMethod"]==item[3]].MonthlyCharges.mean()
print(f"PaymentMethod ({item[0]}/ {item[1]}/ {item[2]}/ {item[3]}) -->  {ave1} / {ave2} / {ave3} / {ave4}.")

plt.tight_layout()
plt.show()

df_data.head()

**Demographic info**
* The monthly charges of Senior Citizen is higher than the others. Peak - Senior Citizen: around 100 dollars, Not Senior Citizen: around 20 dollars.
* The presence of a partner/dependents do not have much effect on the distribution of monthly charges

**Service**
* Customers who did not use PhoneService monthly spent 25 dollars less than those who did, and the distribution of them is centralized.
* The three category of InternetService can be clearly distinguished, customers using Fiber optic pay much more than others. Average - Fiber optic: 91.5, DSL: 58.1, No Service: 21.1.
* There is no particular trend in the distribution of monthly charges by InternetService items, but if customers sign up for the services, the charge will be slightly higher

**Customer account info**
* Customers using paperless bills pay more than others. Peak - PaperlessBilling: around 90 dollars, Not PaperlessBilling: around 20 dollars.
* Customers who left pay more than others. Peak - Churn: around 80 dollars, Not Churn: around 20 dollars.
* Customers who select mail check payment method spent least, and customers who choose the electronic payment method spend the most. The the distribution of monthly charges by Bank transfer/Credit card are almost the same.

**4.3 Composition of different groups(Churn or No Churn)**

In [ ]:
def Donut_Chart(col_name):
    def df(col_name, churn_YoN):
        df = df_data[df_data["Churn"]==churn_YoN][col_name].value_counts().to_frame().reset_index()
        return df

    fig, ax = plt.subplots(1,2, figsize=(18,6))
    dict = {"width": 0.5}

    ax[0].pie(df(col_name, 0)[col_name], 
            colors = ["Salmon", "Gold", "#5F9EA0", "#D3D3D3"], 
            autopct='%1.1f%%', startangle = 90, 
            pctdistance = 0.75,
            wedgeprops=dict,
            textprops={'fontsize': 17, 'weight' : 'heavy'})

    ax[1].pie(df(col_name, 1)[col_name], 
            colors = ["Salmon", "Gold", "#5F9EA0", "#D3D3D3"], 
            autopct='%1.1f%%', startangle = 90, 
            pctdistance = 0.75,
            wedgeprops=dict,
            textprops={'fontsize': 17, 'weight' : 'heavy'})

    ax[0].text(0, 0, 'No Churn', ha="center", fontsize = 17, weight="bold")
    ax[1].text(0, 0, 'Churn', ha="center", fontsize = 17, weight="bold")
    for i in range(2):
        ax[i].legend(loc='upper left', fontsize=15, bbox_to_anchor=(0.9, 0.8), labels = df(col_name,i)["index"])

    plt.suptitle("The Composition of the {} grouped by Churn".format(col_name), fontsize = 30)
    fig.tight_layout()
    plt.show()

col = ['SeniorCitizen', 'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
for i in col:
    Donut_Chart(i)

# 5. Prediction Model

#### 5.1 Procesee Data before Modeling

In [ ]:
df_data.head()

In [ ]:
df_data.drop(columns = ["MonthlyCharges_cut", "TotalCharges_cut"], inplace=True)
for i in df_data.columns:
    if df_data[i].dtypes=='object':
        print(f'{i} : {df_data[i].unique()}')
        print("---------------------------------------------------------------------------")

In [ ]:
df_data.replace('No internet service', "No", inplace=True)
df_data.replace('No phone service', "no", inplace=True)

In [ ]:
df_data["MultipleLines"].replace("no", "No", inplace=True)
for i in df_data.columns:
    if df_data[i].dtypes=='object':
        print(f'{i} : {df_data[i].unique()}')
        print("---------------------------------------------------------------------------")

* There are some features in which categorical variables are more than 2 and they are not "Yes" or "No", we will find and concert it.

In [ ]:
for i in df_data.columns:
    if (len(df_data[i].unique()) > 2) & (df_data[i].dtypes != "int64") & (df_data[i].dtypes != "float64"):
        print(i)

* To deal with such kind of feayures, we are required to use **one-hot** encoding.

In [ ]:
more_than_2 = ['InternetService', 'Contract', 'PaymentMethod']
df_data = pd.get_dummies(data = df_data, columns = more_than_2)
df_data.dtypes

In [ ]:
df_data.shape

In [ ]:
for i in df_data.columns:
    if (df_data[i].dtypes == "float64") | (df_data[i].dtypes == "int64"):
        print(i)

* For the numerical data, we will transfer it by MinMaxScaler.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
large_cols = ["tenure", "MonthlyCharges", "TotalCharges"]
df_data[large_cols] = scaler.fit_transform(df_data[large_cols])
df_data[large_cols].head()

In [ ]:
df_data.head()

* We will ransfer "Yes", "No" to "1", "0".

In [ ]:
two_fea = ['SeniorCitizen' ,'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']
for i in two_fea:
    df_data[i].replace({"No": 0, "Yes": 1}, inplace = True)
df_data.head()

In [ ]:
df_data["gender"].replace({"Female": 1, "Male": 0}, inplace=True)

In [ ]:
# Split dataset into train and test
x = df_data.drop('Churn', axis=1)
y = df_data['Churn']
x.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

**5.2 Using Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

model_LR = LogisticRegression(max_iter = 120, random_state = 0, n_jobs = 20)
#training
model_LR.fit(x_train, y_train)
#predict
predict_LR = model_LR.predict(x_test)
#Calculating accuracy score
LR = round(accuracy_score(y_test, predict_LR)*100, 2)
print(LR)
# classification report 
print("\nClassification Report: \n",classification_report(y_test, predict_LR))

# confusion matrix
cm1 = confusion_matrix(y_test, predict_LR)
sns.heatmap(cm1/np.sum(cm1), annot = True, fmt = '0.2%', cmap='Blues')
plt.title("LR Confusion Matrix", fontsize = 12)
plt.show()

**5.3 Using SVM**

In [ ]:
from sklearn.svm import SVC, LinearSVC
model_svm = SVC(kernel = 'rbf', random_state = 42)
model_svm.fit(x_train, y_train)

#prediction
predict_svm = model_svm.predict(x_test)

#calculate accuracy score
SVM = round(accuracy_score(y_test, predict_svm)*100,2)
print(SVM)

print("\nClassification Report: \n",classification_report(y_test, predict_svm))

cm2 = confusion_matrix(y_test, predict_svm)
sns.heatmap(cm2/np.sum(cm2), annot = True, fmt = '0.2%', cmap='Blues')
plt.title("SVM Confusion Matrix", fontsize = 12)
plt.show()

# 6. Conclusion
Aligning all that with the above analysis, we can draw some conclusion here for all 2 groups:

***

**`THE LEAVING CUSTOMERS (Churn = Yes)`**


> 26.6% of customers in this sample left within the last month. Most of them have **no partners or dependents**. They usually use **fiber optic** internet, and they **don’t sign up for additional services other than Streaming movies/Streaming TV**.
> 
> Most of them sign a **Month-to-month contract**, and leave as soon as the contract expires. **Paperless billing** and **electronic payment** are supported by them, and their **bills are usually higher** than other customers.

**Group's characteristics:**

(a) **Demographic info about customers:**
* 26% are senior citizen
* 50% of men and 50% of women
* 64% have no partner
* 83% have no dependents

(b) **Services:**
* 70% use fiber optic internet, and only 6% of them do not sign up for the Internet
* 65-80% do not use additional internet services such as 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', and 'TechSupport'
* 50% don’t use Streaming movies/ Streaming TV and 40% do.

(c) **Customer account information:**
* 89% of customers sign a Month-to-month contract
* 75% adopt paperless billing
* 57% use electronic payment methods
* Most of them leave after only one month
* Most of their monthly charges are distributed in areas with higher monthly charges, and 60% of their monthly charges are higher than average
* The maximum Total Charges is 8685 dollars, but 47% are concentrated in the range of 0-100 dollars
***
**`THE STAYING CUSTOMERS (Churn = No):`**

> 73.4% of customers in this sample stayed within the last month.

**Group's characteristics:**

(a) **Demographic info about customers:** 
* 13% are senior citizen
* 51% of men and 50% of women
* 47% have no partner
* 66% have no dependents

(b) **Services:**
* Most of them use DSL internet(38%). The number of people selected by the three options is evenly distributed (Fiber optic 35%, No 27%)
* 36-40% do not use additional internet services, and the number of people using the additional services is similar to that of not using the services.

(c) **Customer account information:**
* 43% of customers sign a Month-to-month contract/ 32% sign a Two year contract/ 25% sign a One year contract
* 54% adopt paperless billing
* The proportion of users of the four payment methods is the same (Electronic check/ Bank transfer (automatic)/ Credit card (automatic)/ Mailed check)
* The tenures concentrate around 1 and 72 months, and the number of customers is evenly distributed in other months
* Most of them don't spend much, the monthly charges is concentrated at 20 dollars. (Range: 18-120 dollars)
* Most of their total charges are concentrated in areas with a small number. The higher the total charges, the fewer the number of people.

# 7. Business measures for company
1. SeniorCitizen — A very high percentage of the senior citizens left within the last month (42%). After our research, we found that the senior citizens have the characteristics of high monthly charges and not signing up for services. As a result, we can develop a preferential program for them based on their characteristics.

2. Contract — During the analysis, we found that the longer the contract length, the less the customer will leave, so we can add some preferential items for long-term contracts to attract customers to sign.

3. If customers have the following identities, they are more likely to leave than others: Senior Citizen/ No Partner/ No Dependents. The Churn rate of customers who are senior citizen without partner and dependents is 50%. We found that these types of identities have common behavior — they rarely sign up for additional services such as OnlineSecurity, OnlineBackup. We can advertise for this type of customer.


*`Thanks for watching.`*